In this step, we create a database or reuse one that is already created. We create a table for each data source joining the tables of each source from the formatted zone database.

Before processing any new tables, we remove previously created tables to avoid passing duplicates to the exploitation zone.

In [7]:
import duckdb
import pandas as pd

sources = ["crimesTables", "pricesTables", "economicStatusTables"]

def formattedToTrusted():
    # Connect to formatted database
    conForm = duckdb.connect(database='../../data/formatted/crimesPrices.db')


    # Connect to trusted database
    conTrus = duckdb.connect(database='../../data/trusted/crimesPrices.db')

    # Delete any existing tables
    tables = conTrus.execute("SHOW TABLES").df()
    for table in tables['name']:
        conTrus.execute("DROP TABLE " + table)
    
    for source in sources:
        temp = pd.DataFrame()
        with open("../../data/formatted/"+source+".txt") as f:
            tables = f.readlines()
        
        # If we have new tables to process
        if len(tables) > 0:
            # Read table names files
            for table in tables:
                temp = pd.concat([temp, conForm.table(table.strip()).to_df()])

            # Create a new table with sourceName without "Tables"
            newTable = source[0:-6]
            conTrus.execute('CREATE OR REPLACE TABLE {0} AS SELECT * FROM temp'.format(newTable))

    conForm.close()
    conTrus.close()

formattedToTrusted()

                                               Crime ID    Month  \
0                                                   NaN  2021-03   
1                                                   NaN  2021-03   
2                                                   NaN  2021-03   
3     5b9663273d76d7ca5aa2fd300c4b8a15ccde4d3ba10641...  2021-03   
4                                                   NaN  2021-03   
...                                                 ...      ...   
5417  ddfd2a8865d8898a1305cc6773507bcfdc969c1e6a17f3...  2021-02   
5418  1e092d5eb71a4858e42e9f6e3c14ebca120d97517085cc...  2021-02   
5419  24f2bed3b3e35126b7603c99fac1594306fcb817347791...  2021-02   
5420  75003ea1e2ab148e84ce3efc7f18561ab9db165bd550e4...  2021-02   
5421  d3735f2322583dd8933ca40c008055dca7f43ff7b61f71...  2021-02   

              Reported by         Falls within  Longitude   Latitude  \
0     Lincolnshire Police  Lincolnshire Police   0.068978  52.981910   
1     Lincolnshire Police  Lincolnshire